In [22]:
import numpy as np

from aux_tools import get_text_from_url, get_antiplag_data_from_uid, get_antiplag_uid, get_token_from_file
from typing import NamedTuple, Tuple
from json import loads
import asyncio
from newspaper import Article, ArticleException

In [178]:



class ArticleBaseData(NamedTuple):
    url: str
    title: str
    text: str


class SimilarArticleData(NamedTuple):
    url: str
    similarity_score: float
    similar_words_list: Tuple[str]


class UniqTestResults(NamedTuple):
    date_check: str
    unique_score: float
    cleared_text: str
    similar_articles: Tuple[SimilarArticleData]


class SpellCheckResult(NamedTuple):
    error_type: str
    reason: str
    not_correct_text: str
    correct_replacements: Tuple[str]
    error_segment_start_position: int
    error_segment_end_position: int


class TextKeys(NamedTuple):
    key_title = str
    count = int


class TextKeysGroup(NamedTuple):
    key_title = str
    count = int
    sub_keys = Tuple[TextKeys]


class SeoCheckResult(NamedTuple):
    count_chars_with_space: int
    count_chars_without_space: int
    count_words: int
    water_percent: float
    spam_percent: float
    mixed_words_positions: int
    list_keys: Tuple[TextKeys]
    list_keys_group: Tuple[TextKeysGroup]


class ApTextTestResult(NamedTuple):
    uid: str
    text_unique: float
    uniq_results: UniqTestResults
    spell_results: Tuple[SpellCheckResult]
    seo_results: SeoCheckResult


class TextFeatures(NamedTuple):
    mistakes_count: int
    spam_index: float
    water_index: float
    is_directional_pronouns_used: bool
    is_direct_appear: bool
    is_any_links: bool


class AnalyzedArticle(object):

    def load_semilarity_data(self):
        raw_sem_data = get_antiplag_data_from_uid(self.similarity_data.uid)
        rj = loads(raw_sem_data['result_json'])
        sch = loads(raw_sem_data['spell_check'])
        seo = loads(raw_sem_data['seo_check'])

        self.similarity_data.text_unique = raw_sem_data['text_unique']

        self.similarity_data.uniq_results = UniqTestResults
        self.similarity_data.uniq_results.similar_articles = []
        self.similarity_data.uniq_results.date_check = rj['date_check']
        self.similarity_data.uniq_results.unique_score = rj['unique']
        self.similarity_data.uniq_results.cleared_text = rj['clear_text']
        for element in rj['urls']:
            self.similarity_data.uniq_results.similar_articles.append(
                SimilarArticleData(url=element['url'], similarity_score=element['plagiat'],
                                   similar_words_list=str(element['words']).split(' ')))

        self.similarity_data.spell_results = []
        for element in sch:
            self.similarity_data.spell_results.append(
                SpellCheckResult(error_type=element['error_type'], reason=element['reason'],
                                 not_correct_text=element['error_text'],
                                 correct_replacements=element['replacements'],
                                 error_segment_start_position=element['start'],
                                 error_segment_end_position=element['end']))

        self.similarity_data.seo_results = SeoCheckResult
        self.similarity_data.seo_results.mixed_words_positions = rj['mixed_words']
        self.similarity_data.seo_results.count_chars_with_space = seo['count_chars_with_space']
        self.similarity_data.seo_results.count_chars_without_space = seo['count_chars_without_space']
        self.similarity_data.seo_results.count_words = seo['count_words']
        self.similarity_data.seo_results.water_percent = seo['water_percent']
        self.similarity_data.seo_results.list_keys = []
        # TODO: implement load list keys

        self.similarity_data.seo_results.list_keys_group = []
        # TODO: inplement load list keys group

        self.similarity_data.seo_results.spam_percent = seo['spam_percent']

    def get_semilarity_data(self) -> ApTextTestResult:
        return self.similarity_data

    def start_data_gethering(self):
        self.load_semilarity_data()
        pass

    def __init__(self, article_id: str, article_title: str, article_text: str) -> None:
        self.article_id = article_id
        self.title = article_title
        self.text = article_text

        self.similarity_data = ApTextTestResult
        self.similarity_data.uid = get_antiplag_uid(self.text)['text_uid']

        self.is_primary_source = None  # -   логический признак если первоисточник новости найден
        self.primary_source_url = None  # -   ссылка на первоисточник
        self.source_reliability = None  # -   рейтинг достоверности первоисточника
        self.date_delta = None  # -   дней между размещением первоисточником и этой новостью
        self.difference = None  # -   отличие от первоисточника по тексту (процент "плагиата")
        self.difference_sum = None  # -   отличие текса "саммаризации" статьи от оригинала
        self.deltav_tone_vector = None  # -   изменение вектора тональности
        self.crossed_words = None  # -   список слов статьи, пересекающихся с первоисточником
        self.media_count = None  # -   список ресурсов, разместивших статью
        self.media_avg_rate = None  # -   средний рейтинг ресурсов, разместивших статью
        self.good_media_percentage = None  # -   доля ресурсов с рейтингом доверия более 8 (по 10 бальной системе)
        self.diagram_1 = None  # -   данные по динамике перепечатки статьи
        self.is_real_publication_date = None  # -   указана ли в статье дата публикации
        self.is_publication_date_difference = None  # -   различается ли дата публикации в статье с фактической датой публикации
        self.is_author_shown = None  # -   указан ли в статье автор публикации
        self.real_references = None  # -   ссылки на источник новости (организации, новости, исследователи и тд)
        self.is_organisation_real = None  # -   реальны ли эти организации
        self.author_rate = None  # -   рейтинг автора
        self.mistakes_count = None  # -   количество ошибок в тексте
        self.spam_index = None  # -   индекс "мусора"
        self.water_index = None  # -   индекс "воды"
        self.is_directional_pronouns_used = None  # -   использованы ли указательные местоимения
        self.is_direct_appear = None  # -   использованы ли прямые обращения к читателю
        self.is_any_links = None  #



In [128]:
test_cases =list()
data1_title = 'Москва заняла первое место среди европейских городов в рейтинге инноваций, помогающих в борьбе с COVID-19'
data1_text = '''В мире Москва занимает третье место, уступая лишь Нью-Йорку и Сан-Франциско.
Москва признана первой среди европейских городов в рейтинге инноваций, помогающих в формировании устойчивости коронавирусу. Она опередила Лондон и Барселону.
Среди мировых мегаполисов российская столица занимает третью строчку — после Сан-Франциско и Нью-Йорка. Пятерку замыкают Бостон и Лондон. Рейтинг составило международное исследовательское агентство StartupBlink.
Добиться высоких показателей Москве помогло почти 160 передовых решений, которые применяются для борьбы с распространением коронавируса.
Среди них алгоритмы компьютерного зрения на основе искусственного интеллекта. Это методика уже помогла рентгенологам проанализировать более трех миллионов исследований.
Еще одно инновационное решение — облачная платформа, которая объединяет пациентов, врачей, медицинские организации, страховые компании, фармакологические производства и сайты.
Способствовали высоким результатам и технологии, которые помогают адаптировать жизнь горожан во время пандемии. Это проекты в сфере умного туризма, электронной коммерции и логистики, а также дистанционной работы и онлайн-образования.
Эксперты агентства StartupBlink оценивали принятые в Москве меры с точки зрения эпидемиологических показателей и влияния на экономику.'''
test_cases.append([data1_title, data1_text])

data2_title = 'Москва стала лидером в Европе в рейтинге инноваций, помогающих в борьбе с COVID-19'
data2_text = '''В мире российская столица заняла третье место, обогнав Лондон и Барселону.
Москва заняла первое место среди европейских городов в рейтинге инноваций, помогающих в борьбе с COVID-19, опередив Лондон и Барселону. Об этом сообщает портал мэра и правительства Москвы. В мире российская столица заняла третье место, уступив лишь Нью-Йорку и Сан-Франциско.
В российской столице применяются почти 160 передовых решений для борьбы с распространением коронавируса. Среди них алгоритмы компьютерного зрения на основе искусственного интеллекта, а такжеоблачная платформа, которая объединяет пациентов, врачей, медицинские организации, страховые компании, фармакологические производства и сайты. Способствовали высоким результатам и технологии, которые помогают горожанам адаптироваться во время пандемии. Это проекты в сфере умного туризма, электронной коммерции и логистики, а также дистанционной работы и онлайн-образования.
Рейтинг составляется на базе глобальной карты инновационных решений по борьбе с коронавирусом и оценивает около 100 ведущих городов и 40 стран мира.'''
test_cases.append([data2_title, data2_text])


data3_title = 'Москва заняла первое место в Европе по инновациям в борьбе с COVID-19'
data3_text = '''Москва обошла европейские столицы в рейтинге инноваций по устойчивости к COVID-19, опередив Лондон и Барселону, сообщается на официальном сайте мэра Москвы.
Российская столица также заняла третье место среди мировых мегаполисов. В пятерку лидеров вошли Бостон и Лондон.
Занять лидирующие позиции в рейтинге Москве помогли около 50 передовых решений, которые применяются для борьбы с распространением COVID-19.
Одно из таких решений - алгоритмы компьютерного зрения на основе искусственного интеллекта, которые уже помогли рентгенологам проанализировать более трех миллионов исследований.
Также высоким результатам способствовали технологии, помогающие адаптировать жизнь москвичей во время пандемии. Среди них - проекты в сфере умного туризма, электронной коммерции и логистики, дистанционной работы и онлайн-образования.
Эксперты оценивали, как принятые в Москве меры влияют на эпидемиологические показатели и экономику.'''
test_cases.append([data3_title, data3_text])


data4_title = 'Москва заняла первое место в Европе по инновациям в борьбе с COVID-19'
data4_text = '''Исследовательское агентство StartupBlink составило рейтинг городов по инновациям, помогающим в борьбе с COVID-19. Москва заняла в нем третье место в мире и первое — среди европейских городов.
«Среди мировых мегаполисов российская столица занимает третью строчку — после Сан-Франциско и Нью-Йорка. Пятерку замыкают Бостон и Лондон»,— говорится на сайте московской мэрии.
Среди московских инноваций, используемых для борьбы с коронавирусом,— алгоритмы компьютерного зрения на основе искусственного интеллекта (задействованы в медицинских исследованиях), облачная платформа (объединяет пациентов, врачей, медицинские организации, страховые компании, фармакологические производства и сайты), технологии для адаптации к жизни в городе в период пандемии (проекты в сфере «умного» туризма, электронной коммерции и логистики, а также дистанционной работы и онлайн-образования).
По словам мэра Москвы Сергея Собянина, по состоянию на конец ноября большинство отраслей экономики столицы «чувствуют себя нормально». По данным мэрии, в 2021 году в Москве около 25 тыс. предприятий малого и среднего бизнеса получили поддержку властей.'''
test_cases.append([data4_title, data4_text])


data5_title = 'Москва стала первой в Европе среди городов с инновациями по борьбе с COVID-19'
data5_text = '''Москва заняла первое место среди европейских городов в рейтинге инноваций, которые помогают бороться с коронавирусом. Об этом сообщает пресс-служба мэрии столицы.
Рейтинг составило международное исследовательское агентство «StartupBlink». Среди европейских столиц Москва стала первой, обогнав Лондон и Барселону, а среди мировых городов – третьей, уступив Нью-Йорку и Сан-Франциско.
Как отметили в мэрии Москвы, добиться таких высоких показателей помогли почти 160 передовых инноваций, среди которых алгоритмы компьютерного зрения на основе искусственного интеллекта, облачная платформа, объединившая пациентов, врачей, медицинские организации, страховые компании, фармакологические производства и сайты, а также технологии, которые помогают адаптировать жизнь горожан во время пандемии – проекты в сфере «умного» туризма, электронной коммерции и логистики, дистанционной работы и онлайн-образования.
При этом в правительстве города подчеркнули, что при введении антиковидных ограничений власти отказались от крайностей, сделав ставку на профилактику заболевания – увеличили количество пунктов с бесплатным тестированием и вакцинацией, запатентовали онлайн-программы и платформы для обучения, и занялись развитием телемедицины.'''
test_cases.append([data5_title, data5_text])


data6_title = 'Москва заняла восьмое место среди европейских городов в рейтинге инноваций, помогающих в борьбе с COVID-19'
data6_text = '''В мире Москва занимает лишь восьмое место, уступая в том числе Нью-Йорку и Сан-Франциско.
Москва лишь пятой среди европейских городов в рейтинге инноваций, помогающих в формировании устойчивости коронавирусу. Она пропустила вперёд как Лондон, так и Барселону. Рейтинг составило международное исследовательское агентство StartupBlink.
Добиться высоких показателей Москве помогло почти 70 передовых решений, которые применяются для борьбы с распространением коронавируса.
Среди них алгоритмы компьютерного зрения на основе искусственного интеллекта. Это методика уже помогла рентгенологам проанализировать более пятисот тысяч исследований.
Еще одно инновационное решение — облачная платформа, которая объединяет пациентов, врачей, медицинские организации, страховые компании, фармакологические производства и сайты.
Способствовали высоким результатам и технологии, которые помогают адаптировать жизнь горожан во время пандемии. Это проекты в сфере умного туризма, электронной коммерции и логистики, а также дистанционной работы и онлайн-образования.
Эксперты агентства StartupBlink оценивали принятые в Москве меры с точки зрения эпидемиологических показателей и влияния на экономику.'''
test_cases.append([data6_title, data6_text])


data7_title = 'Бостон занял первое место среди европейских городов в рейтинге инноваций в медицине, а Андроново отказалось от участия в оценке'
data7_text = '''Бостон признан первым среди европейских городов в рейтинге инноваций, помогающих в формировании устойчивости коронавирусу. Он опередил Лондон, Барселону и Андроново.
В мире Бостон занимает третье место, уступая лишь Нью-Йорку и Сан-Франциско. Андроново не участвовало в оценке в этом году. Рейтинг составило международное исследовательское агентство StartupBlink.
Обойти преследователей Бостону помогло более 100 передовых решений, которые применяются для борьбы с распространением коронавируса.
В свою очередь Андроново уже несколько лет не участвует в рейтинге по причине отсутствия кислорода в атмосфере города и водорода в составе воды в реке Лене.
В качестве инновационного решения, позволяющего исправить положение, неким человеком на улице было предложено использовать фаршированных гонобобелем голубей для обеспечения регулярного авиасообщения с планетой Железяка.
Другое предложенное решение оказалось ещё более странным, чем предыдущее — облачная платформа, которая объединяет перистые и кучевые облака в сверхмассивный кластер инновационных перисто-кучевых облаков.
Такого рода высокие технологии вряд ли помогут Андронову занять какое-либо место в каком-нибудь конкурсе.
'''
test_cases.append([data7_title, data7_text])


data8_title = 'Проектный офис ФЭСН РАНХиГС завершает 30 проектов. И начинает новые.'
data8_text = '''Ежегодно в мае-июне Проектный офис Факультета экономических и социальных наук завершает работу над бизнес-проектами и представляет результаты заказчикам – российским и международным компаниям.
Среди компаний-заказчиков были крупные российские компании и организации: Сбер, РЖД, ВЭБ, Минстрой РФ, Ростуризм, Правительство Москвы, киностудия им. Горького, холдинг САВООВ ФУДС, Альфа-Банк, ВТБ, Дом РФ, Очаково, Сегежа-Групп а также представительства зарубежных компаний: BMW, DeLonghi, L’Oreal, Pfizer, Ritter Sport, Xiaomi, Avon, Schneider Group и др.
Проектный офис ФЭСН является крупнейшим университетским проектным центром не только по количеству и сложности проектов, и не только по количеству студентов-участников. Он уже создал и создает новые масштабные форматы проектной работы. Так, еще пять лет назад ФЭСН вовлек в проектную деятельность университеты пяти стран Европы и стал разрабатывать с ними полугодичные проекты для международных компаний на английском языке. Некоторые сложные проекты делались усилиями проектных групп нескольких стран; например, в проекте от генерального директора BMW Russia по «Разработке маркетинговой стратегии перехода компании BMW с бензиновых двигателей на электрические в России» бакалавры ФЭСН работали вместе с магистрами из Германии и Бельгии. А в одном из проектов участвовали команды из Германии, Бельгии, Италии, 4 проектные группы из Франции и 5 из Бразилии.'''
test_cases.append([data8_title, data8_text])

In [149]:
#test_result = AnalyzedArticle(article_id=0,article_title=test_cases[0].title, article_text=test_cases[0].text)
for case in test_cases:
    print(case[0])

Москва заняла первое место среди европейских городов в рейтинге инноваций, помогающих в борьбе с COVID-19
Москва стала лидером в Европе в рейтинге инноваций, помогающих в борьбе с COVID-19
Москва заняла первое место в Европе по инновациям в борьбе с COVID-19
Москва заняла первое место в Европе по инновациям в борьбе с COVID-19
Москва стала первой в Европе среди городов с инновациями по борьбе с COVID-19
Москва заняла восьмое место среди европейских городов в рейтинге инноваций, помогающих в борьбе с COVID-19
Бостон занял первое место среди европейских городов в рейтинге инноваций в медицине, а Андроново отказалось от участия в оценке
Проектный офис ФЭСН РАНХиГС завершает 30 проектов. И начинает новые.


In [107]:
import requests
from bs4 import BeautifulSoup as bs
import re

def get_url_creation_date(url:str):
    template = r'\d{1,2}\s\D*\s\d{4}\s\d{2}\:\d{2}\:\d{2}\s[GMT]{3}'
    webarch_url = 'http://webcache.googleusercontent.com/search?q=cache:'+url+'&strip=1&vwsrc=0'
    raw_text = requests.get(webarch_url).text
    soup = bs(raw_text, 'lxml')
    date_str = soup.find_all('span')[1].text
    result = re.findall(template,date_str)[0]
    return result

'''for i in range(len(results)):
    dt = get_url_creation_date(results[i].similarity_data.uniq_results.similar_articles[0].url)
    print(results[i].similarity_data.uniq_results.similar_articles[0].url, dt)
'''


'for i in range(len(results)):\n    dt = get_url_creation_date(results[i].similarity_data.uniq_results.similar_articles[0].url)\n    print(results[i].similarity_data.uniq_results.similar_articles[0].url, dt)\n'

In [172]:

def get_first_url(uid):
    articles = loads(get_antiplag_data_from_uid(uid)['result_json'])['urls']
    result = list()
    for article in articles:
        result.append([article['url'], get_url_creation_date(article['url'])])
    return result

urls = get_first_url('62a3bcfce38a9')
print(urls)

IndexError: list index out of range

In [179]:
'''uids = list()
for test_case in test_cases:
    uids.append(get_antiplag_uid(test_case[1]))
uids'''

[{'text_uid': '62a3c2880e77c'},
 {'text_uid': '62a3c288922f2'},
 {'text_uid': '62a3c2891c03b'},
 {'text_uid': '62a3c2899a324'},
 {'text_uid': '62a3c28a1de50'},
 {'text_uid': '62a3c28a9c103'},
 {'text_uid': '62a3c28b274bc'},
 {'text_uid': '62a3c28ba845a'}]

In [23]:
uids = [{'text_uid': '62a3c2880e77c'},
 {'text_uid': '62a3c288922f2'},
 {'text_uid': '62a3c2891c03b'},
 {'text_uid': '62a3c2899a324'},
 {'text_uid': '62a3c28a1de50'},
 {'text_uid': '62a3c28a9c103'},
 {'text_uid': '62a3c28b274bc'},
 {'text_uid': '62a3c28ba845a'}]

In [24]:
def get_all_urls(uid):
    result = list()
    urls = loads(get_antiplag_data_from_uid(uid)['result_json'])['urls']
    for url in urls:
        try:
            tmp_url = url['url']
            article = Article(url=tmp_url)
            article.download()
            article.parse()
            result.append([url['url'],article.publish_date])
        except ArticleException:
            pass
    return result

def compile_all_urls(uids):
    result = dict()
    for uid in uids:
        result[uid['text_uid']] = get_all_urls(uid['text_uid'])

    return result

In [25]:
result = compile_all_urls(uids)

In [26]:

result

{'62a3c2880e77c': [['https://innoagency.ru/ru/news/news/moskvazanyalatretiemestovreitinge',
   None],
  ['https://matveychev-oleg.livejournal.com/12400406.html', None],
  ['https://ru.sputnik.kz/20211201/moscow-reyting-antikovidnye-innovatsii-18796711.html',
   datetime.datetime(2021, 12, 1, 0, 0)],
  ['http://mosday.ru/news/item.php?3505963', None],
  ['https://rg.ru/2021/11/30/reg-cfo/moskva-priznana-pervoj-v-evrope-po-innovaciiam-obespechivaiushchim-ustojchivost-k-covid-19.html',
   datetime.datetime(2021, 11, 30, 0, 0)],
  ['https://ria.ru/20211130/reyting-1761553812.html',
   datetime.datetime(2021, 11, 30, 0, 0)],
  ['https://www.mperspektiva.ru/topics/moskva-stala-luchshim-gorodom-evropy-v-reytinge-innovatsiy-po-borbe-s-koronavirusom/',
   datetime.datetime(2021, 1, 12, 9, 0)],
  ['https://xn--80aesfpebagmfblc0a.xn--p1ai/news/20211201-0805.html',
   datetime.datetime(2021, 12, 1, 8, 0)],
  ['https://vz.ru/news/2021/11/30/1131809.html',
   datetime.datetime(2021, 11, 30, 0, 0)],


In [52]:
for key in result.keys():
    for i in range(len(result[key])):
        if len(str(result[key][i][1])) < 5:
            try:
                article = Article(result[key][i][0])
                article.download()
                article.parse()
                result[key][i][1] = article.additional_data
            except ArticleException:
                pass
print(result)

{'62a3c2880e77c': [['https://innoagency.ru/ru/news/news/moskvazanyalatretiemestovreitinge', {}], ['https://matveychev-oleg.livejournal.com/12400406.html', {}], ['https://ru.sputnik.kz/20211201/moscow-reyting-antikovidnye-innovatsii-18796711.html', datetime.datetime(2021, 12, 1, 0, 0)], ['http://mosday.ru/news/item.php?3505963', {}], ['https://rg.ru/2021/11/30/reg-cfo/moskva-priznana-pervoj-v-evrope-po-innovaciiam-obespechivaiushchim-ustojchivost-k-covid-19.html', datetime.datetime(2021, 11, 30, 0, 0)], ['https://ria.ru/20211130/reyting-1761553812.html', datetime.datetime(2021, 11, 30, 0, 0)], ['https://www.mperspektiva.ru/topics/moskva-stala-luchshim-gorodom-evropy-v-reytinge-innovatsiy-po-borbe-s-koronavirusom/', datetime.datetime(2021, 1, 12, 9, 0)], ['https://xn--80aesfpebagmfblc0a.xn--p1ai/news/20211201-0805.html', datetime.datetime(2021, 12, 1, 8, 0)], ['https://vz.ru/news/2021/11/30/1131809.html', datetime.datetime(2021, 11, 30, 0, 0)], ['https://lomonosovskiymedia.ru/news/moskva

In [62]:
import pandas as pd
import numpy as np

result_list = list()
for key in result.keys():
    for i in range(len(result[key])):
        val = result[key][i][1]
        if len(str(result[key][i][1])) < 5:
            val = np.nan
        result_list.append([key, result[key][i][0], val])

print(result_list)

[['62a3c2880e77c', 'https://innoagency.ru/ru/news/news/moskvazanyalatretiemestovreitinge', nan], ['62a3c2880e77c', 'https://matveychev-oleg.livejournal.com/12400406.html', nan], ['62a3c2880e77c', 'https://ru.sputnik.kz/20211201/moscow-reyting-antikovidnye-innovatsii-18796711.html', datetime.datetime(2021, 12, 1, 0, 0)], ['62a3c2880e77c', 'http://mosday.ru/news/item.php?3505963', nan], ['62a3c2880e77c', 'https://rg.ru/2021/11/30/reg-cfo/moskva-priznana-pervoj-v-evrope-po-innovaciiam-obespechivaiushchim-ustojchivost-k-covid-19.html', datetime.datetime(2021, 11, 30, 0, 0)], ['62a3c2880e77c', 'https://ria.ru/20211130/reyting-1761553812.html', datetime.datetime(2021, 11, 30, 0, 0)], ['62a3c2880e77c', 'https://www.mperspektiva.ru/topics/moskva-stala-luchshim-gorodom-evropy-v-reytinge-innovatsiy-po-borbe-s-koronavirusom/', datetime.datetime(2021, 1, 12, 9, 0)], ['62a3c2880e77c', 'https://xn--80aesfpebagmfblc0a.xn--p1ai/news/20211201-0805.html', datetime.datetime(2021, 12, 1, 8, 0)], ['62a3c28

In [148]:
df = pd.DataFrame(result_list, columns=['uid', 'url', 'date'])
df

,uid,url,date
0,62a3c2880e77c,https://innoagency.ru/ru/news/news/moskvazanya...,None
1,62a3c2880e77c,https://matveychev-oleg.livejournal.com/124004...,None
2,62a3c2880e77c,https://ru.sputnik.kz/20211201/moscow-reyting-...,2021-12-01 00:00:00
3,62a3c2880e77c,http://mosday.ru/news/item.php?3505963,None
4,62a3c2880e77c,https://rg.ru/2021/11/30/reg-cfo/moskva-prizna...,2021-11-30 00:00:00
...,...,...,...
198,62a3c28b274bc,https://koronavirus.center/news/moskva-stala-p...,2021-11-30 22:40:21+03:00
199,62a3c28b274bc,https://www.5-tv.ru/news/367794/moskva-stala-p...,NaN
200,62a3c28b274bc,https://sputnik-abkhazia.ru/20211201/moskva-st...,2021-12-01 00:00:00
201,62a3c28ba845a,https://fesn.ranepa.ru/3291,NaN


In [75]:
article = Article(url=df.loc[200]['url'])
article.download()
article.parse()
article.publish_date

In [113]:
for i in df[df.date.isna()].index:
    try:
        df['date'][i] = get_url_creation_date(df['url'][i])
    except:
        pass

In [114]:
df

,uid,url,date
0,62a3c2880e77c,https://innoagency.ru/ru/news/news/moskvazanya...,None
1,62a3c2880e77c,https://matveychev-oleg.livejournal.com/124004...,None
2,62a3c2880e77c,https://ru.sputnik.kz/20211201/moscow-reyting-...,2021-12-01 00:00:00
3,62a3c2880e77c,http://mosday.ru/news/item.php?3505963,13 май 2022 02:55:00 GMT
4,62a3c2880e77c,https://rg.ru/2021/11/30/reg-cfo/moskva-prizna...,2021-11-30 00:00:00
...,...,...,...
198,62a3c28b274bc,https://koronavirus.center/news/moskva-stala-p...,2021-11-30 22:40:21+03:00
199,62a3c28b274bc,https://www.5-tv.ru/news/367794/moskva-stala-p...,None
200,62a3c28b274bc,https://sputnik-abkhazia.ru/20211201/moskva-st...,2021-12-01 00:00:00
201,62a3c28ba845a,https://fesn.ranepa.ru/3291,None


In [123]:
df['date'] = df['date'].apply(str).astype(str)
df

,uid,url,date
0,62a3c2880e77c,https://innoagency.ru/ru/news/news/moskvazanya...,None
1,62a3c2880e77c,https://matveychev-oleg.livejournal.com/124004...,None
2,62a3c2880e77c,https://ru.sputnik.kz/20211201/moscow-reyting-...,2021-12-01 00:00:00
3,62a3c2880e77c,http://mosday.ru/news/item.php?3505963,13 май 2022 02:55:00 GMT
4,62a3c2880e77c,https://rg.ru/2021/11/30/reg-cfo/moskva-prizna...,2021-11-30 00:00:00
...,...,...,...
198,62a3c28b274bc,https://koronavirus.center/news/moskva-stala-p...,2021-11-30 22:40:21+03:00
199,62a3c28b274bc,https://www.5-tv.ru/news/367794/moskva-stala-p...,None
200,62a3c28b274bc,https://sputnik-abkhazia.ru/20211201/moskva-st...,2021-12-01 00:00:00
201,62a3c28ba845a,https://fesn.ranepa.ru/3291,None


In [124]:
df.to_excel('msh_1.xlsx')

In [125]:
df = pd.read_excel('msh_out.xlsx')
df

,Unnamed: 0,uid,url,date
0,6,62a3c2880e77c,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 09:00:00
1,53,62a3c288922f2,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 00:00:00
2,102,62a3c2891c03b,https://www.mos.ru/news/item/96369073/,2021-09-22 00:00:00
3,125,62a3c2899a324,https://www.kommersant.ru/doc/5090994,2021-11-26 00:00:00
4,134,62a3c28a1de50,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 00:00:00
5,163,62a3c28a9c103,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 09:00:00
6,188,62a3c28b274bc,https://www.rosbalt.ru/moscow/2021/11/30/19334...,2021-11-30 00:00:00
7,201,62a3c28ba845a,https://fesn.ranepa.ru/3291,None


In [132]:
df['initial_text'] = None
for i in range(len(test_cases)):
    df['initial_text'][i] = test_cases[i][1]
df

/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_26424/3791531923.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['initial_text'][i] = test_cases[i][1]


,Unnamed: 0,uid,url,date,initial_text,source_text
0,6,62a3c2880e77c,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 09:00:00,"В мире Москва занимает третье место, уступая л...",None
1,53,62a3c288922f2,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 00:00:00,"В мире российская столица заняла третье место,...",None
2,102,62a3c2891c03b,https://www.mos.ru/news/item/96369073/,2021-09-22 00:00:00,Москва обошла европейские столицы в рейтинге и...,None
3,125,62a3c2899a324,https://www.kommersant.ru/doc/5090994,2021-11-26 00:00:00,Исследовательское агентство StartupBlink соста...,None
4,134,62a3c28a1de50,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 00:00:00,Москва заняла первое место среди европейских г...,None
5,163,62a3c28a9c103,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 09:00:00,"В мире Москва занимает лишь восьмое место, уст...",None
6,188,62a3c28b274bc,https://www.rosbalt.ru/moscow/2021/11/30/19334...,2021-11-30 00:00:00,Бостон признан первым среди европейских городо...,None
7,201,62a3c28ba845a,https://fesn.ranepa.ru/3291,None,Ежегодно в мае-июне Проектный офис Факультета ...,None


In [135]:
df['source_text'] = None
for i in range(len(test_cases)):
    article = Article(url=df['url'][i])
    article.download()
    article.parse()
    df['source_text'][i] = article.text
df

/var/folders/gp/7c7n5yqd14z8b8k4499t4r_m0000gn/T/ipykernel_26424/3817476196.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['source_text'][i] = article.text


,Unnamed: 0,uid,url,date,initial_text,source_text
0,6,62a3c2880e77c,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 09:00:00,"В мире Москва занимает третье место, уступая л...",
1,53,62a3c288922f2,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 00:00:00,"В мире российская столица заняла третье место,...",
2,102,62a3c2891c03b,https://www.mos.ru/news/item/96369073/,2021-09-22 00:00:00,Москва обошла европейские столицы в рейтинге и...,Алгоритмы компьютерного зрения на основе искус...
3,125,62a3c2899a324,https://www.kommersant.ru/doc/5090994,2021-11-26 00:00:00,Исследовательское агентство StartupBlink соста...,253 1 мин. ...\n\nБольшинство отраслей экономи...
4,134,62a3c28a1de50,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 00:00:00,Москва заняла первое место среди европейских г...,
5,163,62a3c28a9c103,https://www.mperspektiva.ru/topics/moskva-stal...,2021-01-12 09:00:00,"В мире Москва занимает лишь восьмое место, уст...",
6,188,62a3c28b274bc,https://www.rosbalt.ru/moscow/2021/11/30/19334...,2021-11-30 00:00:00,Бостон признан первым среди европейских городо...,ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ) СОЗДАНО И (ИЛИ) РА...
7,201,62a3c28ba845a,https://fesn.ranepa.ru/3291,None,Ежегодно в мае-июне Проектный офис Факультета ...,


In [136]:
df.to_excel('output.xlsx')

In [138]:
!pip install htmldate

     |████████████████████████████████| 4.6 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 289 kB 2.4 MB/s eta 0:00:01
  Attempting uninstall: regex
    Found existing installation: regex 2022.3.15
    Uninstalling regex-2022.3.15:
      Successfully uninstalled regex-2022.3.15
  Attempting uninstall: lxml
    Found existing installation: lxml 4.6.1
    Uninstalling lxml-4.6.1:
      Successfully uninstalled lxml-4.6.1
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [140]:
from htmldate import find_date

In [150]:
for i in range(df.shape[0]):
    try:
        df['date'][i] = find_date(df['url'][i])
    except ValueError:
        pass
df

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/dateparser/date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


,uid,url,date
0,62a3c2880e77c,https://innoagency.ru/ru/news/news/moskvazanya...,2021-12-01
1,62a3c2880e77c,https://matveychev-oleg.livejournal.com/124004...,2021-12-01
2,62a3c2880e77c,https://ru.sputnik.kz/20211201/moscow-reyting-...,2021-12-01
3,62a3c2880e77c,http://mosday.ru/news/item.php?3505963,2021-12-01
4,62a3c2880e77c,https://rg.ru/2021/11/30/reg-cfo/moskva-prizna...,2021-11-30
...,...,...,...
198,62a3c28b274bc,https://koronavirus.center/news/moskva-stala-p...,2021-11-30
199,62a3c28b274bc,https://www.5-tv.ru/news/367794/moskva-stala-p...,2003-01-07
200,62a3c28b274bc,https://sputnik-abkhazia.ru/20211201/moskva-st...,2021-12-01
201,62a3c28ba845a,https://fesn.ranepa.ru/3291,2022-06-06


In [151]:
df[df.date.isna()]

,uid,url,date
36,62a3c2880e77c,https://tass.ru/obschestvo/13073369,None
64,62a3c288922f2,https://msk.dixinews.ru/news/obshchestvo/moskv...,None
75,62a3c288922f2,https://tass.ru/obschestvo/13073369,None
128,62a3c2899a324,https://tass.ru/obschestvo/13073369,None
129,62a3c28a1de50,https://msk.dixinews.ru/news/obshchestvo/moskv...,None
145,62a3c28a1de50,https://www.m24.ru/videos/gorod/30112021/319056,NaN
170,62a3c28a9c103,https://www.m24.ru/news/gorod/30112021/194371,NaN
175,62a3c28a9c103,https://tass.ru/obschestvo/13073369,None
187,62a3c28a9c103,https://mephi.ru/content/public/uploads/files/...,None
197,62a3c28b274bc,https://tass.ru/obschestvo/13073369,None
